In [ ]:
import pandas as pd
# HTML PARSER
from bs4 import BeautifulSoup as soup
from bs4.element import Tag

# INITIALIZE DRIVER
from selenium import webdriver

# METHODS
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from base import Page

from locators import Login, Rf

import re

### Necessary steps to authenticate credentials and work with Google Sheets

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds']

path = 'c:/Users/rsilva/Gdrive/projects/programming/automation/gspread-91fb9e2e8b49.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(path, scope)

gc = gspread.authorize(credentials)

#### Module to Convert gsheets to pandas df

In [42]:
import gspread_dataframe as gsTodf

In [ ]:
class LoginPage(Page):

    def login(self, usr, pwd):
        """
        make login and return and instance of browser"""

        self.driver.get(Login.URL)
        self.driver.maximize_window()

        usuario = self.wait_for_element_to_click(Login.LOGIN)
        senha = self.wait_for_element_to_click(Login.SENHA)

        # Clear any clutter on the form
        usuario.clear()
        usuario.send_keys(usr)

        senha.clear()
        senha.send_keys(pwd)

        # Hit Enter
        senha.send_keys(Keys.RETURN)

        return PagInicial(self.driver)

In [ ]:
class PagInicial(Page):
    
    def go_to_RF(self):
        self.driver.get(Rf.URL)
        
    def pesquisa_obito(self, cpf):
        
        if self.get_url != Rf.URL:
            with self.wait_for_page_load():
                self.go_to_RF() 
        
        entidade = self.wait_for_element((By.ID, "textfield-1014-inputEl"))
        entidade.clear()
        
        entidade.send_keys(str(cpf) + Keys.RETURN)
        
        try:
            
            elem = self.elem_is_visible((By.CLASS_NAME, "x-grid-cell-inner"))
        
            if "Óbito" in str(elem.text):
                return "OBITO"
            else:
                return "ATIVO"

        except:
            print("Problema ao resgatar o elemento da busca")
            return ""

        
    def pesquisa_dados(self, cpf):
        
        if self.get_url != Rf.URL:
            with self.wait_for_page_load():
                self.go_to_RF() 
        
        entidade = self.wait_for_element(Rf.IDINPUTCPF)
        entidade.clear()
        
        entidade.send_keys(str(cpf) + Keys.RETURN)
        
        if self.check_element_exists(Rf.RESULTADO):
            
            html = soup(self.driver.page_source, "lxml")
            
            resultado = html.find('tr', {'data-recordid' : str(cpf)})
            
            try:
                resultado = [content.div for content in resultado.contents]
            except:
                print('Resultado não possui atributos contents')
                
            return resultado
        
        else:
            
            return None

In [ ]:
def tags_to_string(lista_tags):
    
    reg = []
    
    for tag in lista_tags:
        
        for string in tag.stripped_strings:
            
            reg.append(string)
            
    # Corrige o registro de nome e endereço    
    reg[0] = reg[0] + reg[1]
    
    reg.remove(reg[1])
    
    reg[1] = reg[1] + reg[2]
    
    reg.remove(reg[2])
    
    return reg
    

In [ ]:
def cria_dict_dados(registros):
    
    dados = {}
    
    chaves = ['CPF', 'Nome', 'Mãe', 'Data de Nascimento', 'Sexo', 
                 'Ano do Óbito','Nacionalidade','Título de Eleitor', 
                 'Situação Cadastral','Fone']
    
    for s in registros:
    
        s = s.split(':')

        if s[0] in chaves:

            dados[s[0]] = s[1].lstrip(' ')
            
    endereço = registros[-2].split(',')
    
    if len(endereço) == 6:
    
        items_end = ['Logradouro', 'Número', 'Complemento', 'Bairro', 'Cidade',
                'Cep']
    else:
    
        items_end = ['Logradouro', 'Número', 'Bairro', 'Cidade', 'Cep']
    
    
    for k,v in zip(items_end, endereço):

        dados[k] = v.lstrip(' ')
        
    # corrige CEP 
    
    if 'Cep' in dados:
        dados['Cep'] = dados['Cep'][-9:]
        
    return dados

### Código para logar no Sapiens

In [ ]:
driver = webdriver.Chrome()
sapiens = LoginPage(driver).login('33831640807', 'Fer465023@ur_')

### Criar o Data Frame para exportar os dados

In [ ]:
keys = ['CPF', 'Nome', 'Mãe', 'Data de Nascimento', 'Sexo', 
        'Ano do Óbito','Nacionalidade','Título de Eleitor', 
        'Situação Cadastral','Fone', 'Logradouro', 'Número', 
        'Complemento', 'Bairro', 'Cidade','Cep']

df = pd.DataFrame(columns = keys)

### Leitura dos Dados Brutos na Planilha

In [ ]:
sht = gc.open(title='RC').worksheet('Registros')
lista_CPF = sht.col_values(1)[1:]

In [ ]:
dados = pd.DataFrame()

for CPF in lista_CPF[400:]:
    
    resultado = cria_dict_dados(tags_to_string(sapiens.pesquisa_dados(CPF)))
    
    df = df.append(pd.Series(resultado), ignore_index=True)
    
df.tail()

In [44]:
plan_RF = gc.open(title='RC').worksheet('Atualização')

In [45]:
gsTodf.set_with_dataframe(plan_RF, df)

In [ ]:
planilha = pd.read_table("planilha_obito_guilherme.tsv", converters={'CPF': lambda x: str(x), 'FISTEL': lambda x: str(x)})

In [46]:
IE = webdriver.Ie()

WebDriverException: Message: Unexpected error launching Internet Explorer. Protected Mode settings are not the same for all zones. Enable Protected Mode must be set to the same value (enabled or disabled) for all zones.
